In [1]:
import weaviate
from weaviate.auth import Auth
import weaviate.classes as wvc

WEAVIATE_URL = "https://1tnwq0iequmnhcrcultoq.c0.us-east1.gcp.weaviate.cloud"
WEAVIATE_API_KEY = "pV6haMawVUuFMWiu8RGRkbrpjo4YfGehXHmV"
collection_name = "HolocaustTestimonies"

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)

holocaust_testimonies = client.collections.get(name=collection_name)

In [2]:
from sentence_transformers import SentenceTransformer
from weaviate.classes.query import MetadataQuery

model = SentenceTransformer('sentence-transformers/LaBSE')

/Applications/anaconda3/envs/fasthtml/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Applications/anaconda3/envs/fasthtml/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Applications/anaconda3/envs/fasthtml/lib/python3.10/site-packages/sentence_transformers/models/Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to cons

In [3]:
def find_similar(query, threshold):
    query_vector = model.encode([query])[0]
    response = holocaust_testimonies.query.near_vector(
        near_vector=query_vector,
        limit=10,
        return_metadata=MetadataQuery(distance=True)
    )
    print(response)

    for o in response.objects:
        if o.metadata.distance < threshold:
            print(o.properties["book"], o.properties["chapter"], o.properties["verse"], ": ", o.properties["text"])
            print(o.metadata.distance)